In [36]:
import requests
from datetime import datetime

In [76]:
class disney_park:
    def __init__(self):
        self.tokendata = self.get_tokendata()
        self.token = self.tokendata['access_token']
        self.expirtime = self.tokendata['expires_in']
        self.parkid = self.getparkid()
        self.resortid = self.getresortid()
        self.data = self.get_data()
    
    
    def get_tokendata(self):
        TOKEN_URL = 'https://authorization.go.com/token?grant_type=assertion&assertion_type=public&client_id=WDPRO-MOBILE.MDX.WDW.ANDROID-PROD'
        r = requests.post(url = TOKEN_URL)
        data = r.json()
        return(data)
    
    def getparkid(self):
        raise("Method Must Be Defined By Inherited Class")

        
    def getresortid(self):
        raise("Method Must Be Defined By Inherited Class")
        
    def get_data(self):
        headers = {
        'Accept-Language' : 'en_US',
        'User-Agent': 'UIEPlayer/2.1 iPhone OS 6.0.1',
        'Accept' : 'application/json;apiversion=1',
        'Authorization' : "BEARER "+str(self.token),
        'X-Conversation-Id' : 'WDPRO-MOBILE.MDX.CLIENT-PROD',
        'X-Correlation-ID' : str(datetime.now().timestamp())

        }
        r = requests.get(url = 'https://api.wdpro.disney.go.com/facility-service/theme-parks/{}/wait-times'.format(self.parkid),headers=headers)
        data = r.json()
        return(data)





In [78]:
class Disneyland(disney_park):
    def __init__(self):
        super().__init__()
        
    def getparkid(self):
        return(330339)

        
    def getresortid(self):
        return(80008297)
    
    